In [1]:
import os
import gc
import math
import json
from datetime import datetime
import shutil
from pathlib import Path
import optuna
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from tqdm import tqdm
import torch
from sklearn.model_selection import StratifiedGroupKFold
from torch.utils.data import DataLoader
from typing import Dict, List, Tuple, NamedTuple
from transformers import AutoTokenizer, AutoConfig, BertTokenizerFast
import scml

In [2]:
class ModelConf(NamedTuple):
    directory: Path
    model_max_length: int
    stride: int
    batch_size: int
    gradient_checkpointing: bool


class Conf(NamedTuple):
    model_name: str = "roberta-base"
    epochs: int = 3
    lr: Tuple[float, float] = (2e-6, 2e-6)
    multi_sample_dropout_size: Tuple[int, int] = (8, 8)
    multi_sample_dropout_increment: Tuple[float, float] = (0, 0)
    swa_start_epoch: Tuple[int, int] = (-1, -1)
    swa_anneal_epochs: Tuple[int, int] = (3, 3)
    pretrained_dir: Path = Path("pretrained")
    zoo: Dict[str, ModelConf] = {
        "deberta-v3-base": ModelConf( # sentencepiece tokenizer
            directory=pretrained_dir / "microsoft" / "deberta-v3-base",
            model_max_length=512,
            stride=128,
            batch_size=2,
            gradient_checkpointing=False
        ),
        "roberta-base": ModelConf( # bbpe tokenizer
            directory=pretrained_dir / "roberta-base",
            model_max_length=512,
            stride=128,
            batch_size=4,
            gradient_checkpointing=False
        )
    }
    sample_frac: float = 1.0
    gpus: List[int] = [0]
    patience: int = 1
    n_trials: int = 1
    n_folds: int = 3
    seed: int = 31
        
        
conf = Conf()
mc = conf.zoo[conf.model_name]
print(conf)

Conf(model_name='roberta-base', epochs=3, lr=(2e-06, 2e-06), multi_sample_dropout_size=(8, 8), multi_sample_dropout_increment=(0, 0), swa_start_epoch=(-1, -1), swa_anneal_epochs=(3, 3), pretrained_dir=WindowsPath('pretrained'), zoo={'deberta-v3-base': ModelConf(directory=WindowsPath('pretrained/microsoft/deberta-v3-base'), model_max_length=512, stride=128, batch_size=2, gradient_checkpointing=False), 'roberta-base': ModelConf(directory=WindowsPath('pretrained/roberta-base'), model_max_length=512, stride=128, batch_size=4, gradient_checkpointing=False)}, sample_frac=1.0, gpus=[0], patience=1, n_trials=1, n_folds=3, seed=31)


In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(conf.seed)
pl.seed_everything(conf.seed)

Global seed set to 31


31

In [4]:
device: torch.device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

cpu


In [5]:
%%time
df = pd.read_parquet("input/sequences.parquet")
if conf.sample_frac<1:
    df = df.sample(frac=conf.sample_frac)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12899779 entries, 0 to 12899778
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   session  int32 
 1   seq      object
 2   length   int16 
dtypes: int16(1), int32(1), object(1)
memory usage: 172.2+ MB
Wall time: 12.3 s


In [6]:
#tokenizer = AutoTokenizer.from_pretrained(str(mc.directory), model_max_length=mc.model_max_length)
tokenizer = BertTokenizerFast(
    vocab_file="input/vocab.txt", 
    unk_token="<unk>",
    sep_token="<s>",
    pad_token="<pad>",
    cls_token="<cls>",
    mask_token="<mask>",
    bos_token="<s>",
    eos_token="</s>",
    additional_special_tokens=["click_token", "cart_token", "order_token"],
    model_max_length=mc.model_max_length,
    padding_side="right",
)
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<click>", "<cart>", "<order>"],
})
unk_token = tokenizer.unk_token
unk_id = tokenizer.unk_token_id
pad_token = tokenizer.pad_token
pad_id = tokenizer.pad_token_id
sep_token = tokenizer.sep_token
sep_id = tokenizer.sep_token_id
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"{unk_token}={unk_id}\n{pad_token}={pad_id}\n{sep_token}={sep_id}")

PreTrainedTokenizerFast(name_or_path='', vocab_size=1855612, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<s>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>', 'additional_special_tokens': ['<click>', '<cart>', '<order>']})
model_input_names=['input_ids', 'token_type_ids', 'attention_mask']
<unk>=3
<pad>=1
<s>=0


In [ ]:
%%time
x = tokenizer(
    df["seq"].tolist(),
    truncation=True, 
    padding="max_length",
    stride=mc.stride,
    add_special_tokens=True,
    return_overflowing_tokens=True,
    return_offsets_mapping=False,
    return_special_tokens_mask=False,
)
print(f"{repr(x.keys())}")
overflow_to_sample_mapping = x["overflow_to_sample_mapping"]
print(f"len(overflow_to_sample_mapping)={len(overflow_to_sample_mapping):,}")
#offset_mapping = x["offset_mapping"]
#print(f"len(offset_mapping)={len(offset_mapping):,}")
print(x["input_ids"][0])

In [ ]:
assert x["input_ids"][0] == []

In [9]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")
#print(f"Saved {str(job_dir)}")

Total time taken 0:00:15.738479
